In [1]:
def bootstrap():
    dir = %pwd
    if dir.endswith('/bertology-workshop'):
        # we're set
        pass
    elif dir.endswith('bertology-workshop/notebooks'):
        # looks like we're inside jupyter
        %cd ..
    elif dir == '/content':
        # looks like we're on colab
        !git clone git://github.com:diunko/bertology-workshop
        %cd bertology-workshop
    else:
        raise RuntimeError('I have to be launched from colab or from '
                           'bertology-workshop/notebooks')
    dir = %pwd
    return dir
bootstrap()

/dvl/nlp/learning/bertology-workshop


'/dvl/nlp/learning/bertology-workshop'

In [2]:
!make requirements > /dev/null

### Download GLUE data

In [3]:
!make data/glue

make: Nothing to be done for `data/glue'.


In [2]:
import os
import time
import glob

import torch

from bertology.glue import GLUETransformer
from bertology.glue import generic_train
from bertology.utils import args_to_dict, dict_to_args

INFO:transformers.file_utils:PyTorch version 1.5.0 available.


In [6]:
GLUETransformer.from_dict(dict(help=True))

usage: ipykernel_launcher.py [-h] [--output_dir OUTPUT_DIR] [--fp16]
                             [--fp16_opt_level FP16_OPT_LEVEL] [--n_gpu N_GPU]
                             [--n_tpu_cores N_TPU_CORES]
                             [--max_grad_norm MAX_GRAD_NORM] [--fast_dev_run]
                             [--do_train] [--do_predict]
                             [--gradient_accumulation_steps GRADIENT_ACCUMULATION_STEPS]
                             [--seed SEED] --model_name_or_path
                             MODEL_NAME_OR_PATH [--config_name CONFIG_NAME]
                             [--tokenizer_name TOKENIZER_NAME]
                             [--cache_dir CACHE_DIR]
                             [--learning_rate LEARNING_RATE]
                             [--weight_decay WEIGHT_DECAY]
                             [--adam_epsilon ADAM_EPSILON]
                             [--warmup_steps WARMUP_STEPS]
                             [--num_train_epochs NUM_TRAIN_EPOCHS]
          

SystemExit: 0

/dvl/nlp/learning/bertology-workshop/.pyenv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
hparams = dict(
    model_name_or_path='bert-base-cased',
    data_dir='data/glue/MRPC',
    n_gpu=-1,
    train_batch_size=1,
    task='mrpc',
    fast_dev_run=True,
)

model = GLUETransformer.from_dict(hparams)

args = model.hparams
args_to_dict(model.hparams)

INFO:bertology.glue:hparams: {
  "output_dir": null,
  "fp16": false,
  "fp16_opt_level": "O1",
  "n_gpu": -1,
  "n_tpu_cores": 0,
  "max_grad_norm": 1.0,
  "fast_dev_run": true,
  "do_train": false,
  "do_predict": false,
  "gradient_accumulation_steps": 1,
  "seed": 42,
  "model_name_or_path": "bert-base-cased",
  "config_name": "",
  "tokenizer_name": "",
  "cache_dir": "",
  "learning_rate": 5e-05,
  "weight_decay": 0.0,
  "adam_epsilon": 1e-08,
  "warmup_steps": 0,
  "num_train_epochs": 3,
  "train_batch_size": 1,
  "eval_batch_size": 32,
  "max_seq_length": 128,
  "task": "mrpc",
  "data_dir": "data/glue/MRPC",
  "overwrite_cache": false
}
INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json from cache at /Users/diunko/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391
INFO:transformers

{'output_dir': None,
 'fp16': False,
 'fp16_opt_level': 'O1',
 'n_gpu': -1,
 'n_tpu_cores': 0,
 'max_grad_norm': 1.0,
 'fast_dev_run': True,
 'do_train': False,
 'do_predict': False,
 'gradient_accumulation_steps': 1,
 'seed': 42,
 'model_name_or_path': 'bert-base-cased',
 'config_name': '',
 'tokenizer_name': '',
 'cache_dir': '',
 'learning_rate': 5e-05,
 'weight_decay': 0.0,
 'adam_epsilon': 1e-08,
 'warmup_steps': 0,
 'num_train_epochs': 3,
 'train_batch_size': 1,
 'eval_batch_size': 32,
 'max_seq_length': 128,
 'task': 'mrpc',
 'data_dir': 'data/glue/MRPC',
 'overwrite_cache': False,
 'glue_output_mode': 'classification'}

### Train model

In [4]:
if args.output_dir is None:
    args.output_dir = os.path.join("./runs", f"{args.task}_{time.strftime('%Y%m%d_%H%M%S')}",)
    os.makedirs(args.output_dir)

trainer = generic_train(model, args)

INFO:lightning:Running in fast_dev_run mode: will run a full train, val and test loop using a single batch


In [5]:
trainer.fit(model)

INFO:bertology.glue:Loading features from cached file data/glue/MRPC/cached_train_bert-base-cased_128
INFO:bertology.glue:Loading features from cached file data/glue/MRPC/cached_dev_bert-base-cased_128
INFO:lightning:
    | Name                                                   | Type                          | Params
-----------------------------------------------------------------------------------------------------
0   | model                                                  | BertForSequenceClassification | 108 M 
1   | model.bert                                             | BertModel                     | 108 M 
2   | model.bert.embeddings                                  | BertEmbeddings                | 22 M  
3   | model.bert.embeddings.word_embeddings                  | Embedding                     | 22 M  
4   | model.bert.embeddings.position_embeddings              | Embedding                     | 393 K 
5   | model.bert.embeddings.token_type_embeddings            | Embed

1

### Get metrics from the last checkpoint

In [6]:
checkpoints = list(sorted(glob.glob(
    os.path.join(args.output_dir, "checkpointepoch=*.ckpt"), recursive=True)))

In [10]:
model = GLUETransformer.load_from_checkpoint(checkpoints[-1])
trainer.test(model)


INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json from cache at /Users/diunko/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391
INFO:transformers.configuration_utils:Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 28996
}

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json from cache at /Users/diunko/.c

--------------------------------------------------------------------------------
TEST RESULTS
{'acc': 0.28125,
 'acc_and_f1': 0.140625,
 'avg_test_loss': 0.796065628528595,
 'f1': 0.0,
 'val_loss': 0.796065628528595}
--------------------------------------------------------------------------------

